<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Pandas per la gestione dei dati
import pandas as pd

from collections import Counter

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# Importiamo il dataset ottenuto tramite le operazioni di scraping
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/charting-m-points-2010s.csv")

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329842,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5.0,AD-40,19,True,1,4d,4b3n#,NaN,1
329843,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5.0,0-0,20,True,2,5d,6f1w#,NaN,2
329844,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5.0,15-0,20,True,2,5s38b2f1f1f2b2b2f1f3b3b+1n@,NaN,NaN,2
329845,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5.0,30-0,20,True,2,4f18f1f3b3s3b3s3f+1l1o3*,NaN,NaN,2


Analisi delle tipologie degli esiti finali dei punti, analisi dei casi rari

In [4]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]

    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 110288
'@': 103774
'*': 102804
'd': 4828
'n': 4780
'w': 1397
'S': 1003
'R': 429
'x': 297
'C': 135
'!': 22
' ': 16
'g': 14
'Q': 12
'1': 10
'3': 9
'e': 6
'P': 5
'9': 4
'2': 2
'c': 2
'8': 2
'7': 2
'p': 1
'^': 1
'+': 1
'.': 1
'-': 1
'b': 1


Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile.

La descrizione in formato semplice, una riga per ogni stringa:


- `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

- `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.


Attualmente non ho aggiunto considerazioni sul primo e secondo servizio.

Inoltre non ho considerato posizionamenti della palla e tipologia di colpi giocati.

In [5]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_ace_1'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = 'serve_ace_2'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_miss2_1'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_miss1_2'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = 'noinfo_1'
                else:
                  dataset.at[index, 'PointType'] = 'noinfo_2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_winner_1'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_winner_2'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_forced2_1'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_unforced1_2'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_unforced2_1'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_forced1_2'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = 'noinfo_1'
                else:
                  dataset.at[index, 'PointType'] = 'noinfo_2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_ace_1'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = 'serve_ace_2'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_miss2_1'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = 'serve_miss1_2'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = 'noinfo_1'
                else:
                  dataset.at[index, 'PointType'] = 'noinfo_2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_winner_1'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_winner_2'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_forced2_1'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_unforced1_2'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'rally_unforced2_1'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'rally_forced1_2'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = 'noinfo_1'
                else:
                  dataset.at[index, 'PointType'] = 'noinfo_2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,PointType
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1,rally_unforced2_1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1,rally_unforced2_1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1,serve_ace_1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1,rally_forced2_1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1,rally_unforced2_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329842,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5.0,AD-40,19,True,1,4d,4b3n#,NaN,1,rally_forced2_1
329843,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5.0,0-0,20,True,2,5d,6f1w#,NaN,2,rally_unforced1_2
329844,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5.0,15-0,20,True,2,5s38b2f1f1f2b2b2f1f3b3b+1n@,NaN,NaN,2,rally_forced1_2
329845,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5.0,30-0,20,True,2,4f18f1f3b3s3b3s3f+1l1o3*,NaN,NaN,2,rally_winner_2


In [6]:
noinfo_count = dataset[dataset['PointType'].isin(['noinfo_1', 'noinfo_2'])].shape[0]
print(noinfo_count)

7979


Usare le informazioni precedenti per creare il vettore della progressione dei punti per ogni game accompagnato con un vettore avente la progressione dei game(questo servirà per predire la vittoria del set).

Inoltre servirà aggiungere 3 etichette, vincitore game, vincitore set e vincitore partita che serviranno successivamente.



COMPLETARE LOGICA DELLA PROGRESSIONE DEL SET, QUINDI DI COME SONO STATI VINTI I GAME. POTREBBE SERVIRE PER PREDIRE LA VITTORIA DEL SET

INFINE AGGIUNGERE LA LOGICA SULLE TIPOLOGIE DI PUNTI DEFINITE PRECEDENTEMENTE quindi con PoweredGmProgression

In [10]:
new_data = []
current_id = None
current_set = None
current_game = None
last_point = None
new_row = None

for index, row in dataset.iterrows():
    match_id = row['match_id']
    set_score = row['Set1'] + row['Set2']
    game_score = row['Gm1'] + row['Gm2']
    point_winner = row['PtWinner']

    # New match
    if match_id != current_id:
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point
              if r['MatchWinner'] == 0:
                  r['MatchWinner'] = last_point


        current_id = match_id
        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # New set
    elif set_score != current_set:
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point



        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # New game
    elif game_score != current_game:
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)


        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # Same game
    else:
        if new_row:
            new_row['GmProgression'].append(point_winner)

    last_point = point_winner

# Last dataset's game
if new_row and last_point:
    new_row['GmWinner'] = last_point
    new_data.append(new_row)
    for r in new_data:
      if r['SetWinner'] == 0:
          r['SetWinner'] = last_point
      if r['MatchWinner'] == 0:
          r['MatchWinner'] = last_point




new_dataset = pd.DataFrame(new_data)
new_dataset

,match_id,GmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]",1,0,0,0,0,1,1,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 2, 2, 1, 2, 2, 2]",2,0,0,1,0,2,1,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[2, 1, 1, 1, 1]",1,0,0,1,1,1,1,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 2, 2, 1, 1, 2, 2, 2]",2,0,0,2,1,2,1,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]",1,0,0,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...
53331,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 2, 1, 1, 2, 2]",2,0,1,2,3,2,2,2
53332,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 1, 2, 2, 1]",1,0,1,2,4,1,2,2
53333,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 1, 2, 1, 2, 2]",2,0,1,3,4,2,2,2
53334,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 2, 2, 2, 1, 1, 1]",1,0,1,3,5,1,2,2
